In [1]:
import os
os.environ["LANGCHAIN_TRACING"] = "true" # If you want to trace the execution of the program, set to "true"

from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, load_tools

from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser, # A synchronous browser is available, though it isn't compatible with jupyter.
)

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

In [2]:
def get_input() -> str:
  """Placeholder for Slack input from user."""
  print("Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.")
  contents = []
  while True:
    try:
      line = input()
    except EOFError:
      break
    if line == "q":
      break
    contents.append(line)
  return "\n".join(contents)


In [3]:
# MEMORY 
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [4]:
# LLM 
llm = ChatOpenAI(temperature=0, model="gpt-4-0613", max_retries=3, request_timeout=60 * 3)

In [5]:
# TOOLS
async_browser = create_async_playwright_browser()
browser_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = browser_toolkit.get_tools()
tools

human_tools = load_tools(
    ["human"],
    llm=llm,
    input_func=get_input
)

In [6]:
tools = tools + human_tools
tools

[ClickTool(name='click_element', description='Click on an element with the given CSS selector', args_schema=<class 'langchain.tools.playwright.click.ClickToolInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, sync_browser=None, async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/kastanday/Library/Caches/ms-playwright/chromium-1055/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=112.0.5615.29>, visible_only=True, playwright_strict=False, playwright_timeout=1000),
 NavigateTool(name='navigate_browser', description='Navigate a browser to the specified URL', args_schema=<class 'langchain.tools.playwright.navigate.NavigateToolInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, sync_browser=None, async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/kastanday/Library/Ca

In [7]:
agent_chain = initialize_agent(
  tools, 
  llm, 
  agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
  verbose=True,
  memory=memory, 
  agent_kwargs = {
    "memory_prompts": [chat_history],
    "input_variables": ["input", "agent_scratchpad", "chat_history"]
  }
)

In [10]:
# asycn and sync together is tough...

response = await agent_chain.arun(input="Ask the user what they're interested in learning about on Langchain, then Browse to blog.langchain.dev and summarize the text especially whatever is relevant to the user, please.")
print(response)



> Entering new AgentExecutor chain...
Thought: Before I can browse and summarize the text from the blog, I need to know what the user is interested in learning about on Langchain. I will ask the user for this information.

Action:
```
{
  "action": "human",
  "action_input": {
    "query": "What specific topics are you interested in learning about on Langchain?"
  }
}
```

What specific topics are you interested in learning about on Langchain?
Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.

Observation: agents and structured tools
Thought:The user is interested in learning about "agents" and "structured tools" on Langchain. Now, I will navigate to the Langchain blog to find relevant information.

Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://blog.langchain.dev"
  }
}
```
Observation: Navigating to https://blog.langchain.dev returned status code 200
Thought:Now that I'm on the Langchain blog, I need to extract the tex

The extracted elements contain the full text of the webpage. I can see that there are several mentions of "agents" and "structured tools" in the text. I will now summarize the relevant parts of the text.

The blog post titled "Plan-and-Execute Agents" introduces a new type of agent executor called "Plan-and-Execute", which contrasts against the previous types of agents, referred to as "Action" agents. Plan-and-Execute agents are heavily inspired by BabyAGI and the recent Plan-and-Solve paper.

In the post titled "Structured Tools", a new abstraction is introduced to allow for the usage of more complex tools. While previous tools took in a single string input, new tools can take in an arbitrary number of inputs of arbitrary types. A new agent class that works well with these structured tools is also introduced.

The post "GPTeam: A multi-agent simulation" highlights novel applications of LangChain. After the Generative Agents paper was released, there was a rush of open source projects 

/var/folders/wg/sry3qhhx5ls2vpwn1y7_b3p40000gn/T/ipykernel_42313/2953221896.py:3: RuntimeWarning: coroutine 'Chain.arun' was never awaited
  response = await agent_chain.arun(input="Ask the user what they're interested in learning about on Langchain, then Browse to blog.langchain.dev and summarize the text especially whatever is relevant to the user, please.")


In [ ]:
The LangChain blog contains several posts relevant to your interests. The post titled 'Plan-and-Execute Agents' introduces a new type of agent executor called 'Plan-and-Execute', which contrasts against the previous types of agents, referred to as 'Action' agents. In the post titled 'Structured Tools', a new abstraction is introduced to allow for the usage of more complex tools. The post 'GPTeam: A multi-agent simulation' highlights novel applications of LangChain, indicating a strong interest in agent-based applications. The post 'Unleashing the power of AI Collaboration with Parallelized LLM Agent Actor Trees' discusses the potential of multiple agents working in parallel. This suggests that there is a lot of ongoing research and development in the area of agents and structured tools on LangChain.


In [ ]:
response = await agent_chain.arun(input="Browse to blog.langchain.dev and summarize the text, please.")
print(response)